In [0]:
!pip install pythainlp ssg 

     |████████████████████████████████| 11.1MB 381kB/s 
     |████████████████████████████████| 481kB 55.9MB/s 
     |████████████████████████████████| 1.5MB 56.0MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 757kB 57.1MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449905 sha256=953514ad662742c36f9307fe09e5691fc9a6c89aa613988c92ff5f6220471e86
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103528 sha256=875246a50f21ef90d756c1f3f28334b9ebaa949bd486f59680b25f8bb466a2e4
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
Successfully built nltk fire
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.23.0 which is incompatible.
ERROR: dat

In [1]:
!pwd

/content


In [0]:
from pythainlp import subword_tokenize,word_tokenize
from pythainlp.corpus.common import thai_stopwords,thai_words,thai_syllables,countries,thai_negations,provinces
import re
import heapq
import math


In [0]:

def get_leading_character(word_list):
  leadings=''
  for word in word_list:
    leadings=leadings + extract_leading_character(word)
  return leadings
def extract_leading_character(word):
  word = word.strip()
  word = remove_silent_alphabet(word)
  word = remove_tone(word)
  leading_char = ''
  if (len(word) > 0 and re.match(r".*[ก-ฮ]",word)):
    if re.match(r"[แโใไเ]?(หย|หร|หล|หว|หง|หญ|หน|หม|อย)[ก-๙]+",word):
      leading_char = get_first_two_alphabet(word)[1]
    elif re.match(r"[แโใไเ]?(หล)$",word):
      leading_char = get_first_two_alphabet(word)[1]  
    elif re.match(r"[แโใไเ]?(หย|หร|หว|หง|หญ|หน|หม|อย)$",word):
      leading_char = get_first_two_alphabet(word)[0]
    else:
      leading_char =  get_first_alphabet(word)
  return leading_char
def get_all_alphabet(word):
  return re.findall(r"[ก-ฮ]{1}", word)
def get_first_alphabet(word):
  return get_all_alphabet(word)[0]
def get_first_two_alphabet(word):
  return re.findall(r"[ก-ฮ]{2}", word)[0]
def remove_silent_alphabet(word):
  silent_pattern = r'ธิ์|ธุ์|ตร์|ทร์|ดิ์|ดร์|[ก-ฮ]์'
  return re.sub(silent_pattern, '', word)
def remove_tone(word): # fail case แห่ง แหง
  tone_pattern = r'่|้|๊|๋'
  return re.sub(tone_pattern, '', word)

def replace_um(word):
  word = word.replace('ํา','ำ')
  while 'ํ' in word:
    if word.index('ํ')+2 < len(word) and word[word.index('ํ')+2] == 'า':
      replacing_index= word.index('ํ')+2
      word = word[:replacing_index] + 'ำ' + word[replacing_index+1:]
      replacing_index = replacing_index - 2
      word = word[:replacing_index] + word[replacing_index+1:] 
  return word

def extract_abbreviation(word):
  return get_leading_character(subword_tokenize(word, engine="ssg"))

In [0]:
def run_test_extract_leading_char():
  assert extract_leading_character('หงส์') == 'ห', 'หงส์ returns '+extract_leading_character('หงส์')
  assert extract_leading_character('โหน') == 'ห', 'โหน returns '+extract_leading_character('โหน')
  assert extract_leading_character('โหร') == 'ห', 'โหร returns '+extract_leading_character('โหร')
  assert extract_leading_character('โหล') == 'ล', 'โหล returns '+extract_leading_character('โหล')
  assert extract_leading_character('หน') == 'ห', 'หน returns '+extract_leading_character('หน')
  assert extract_leading_character('เหง') == 'ห', 'เหง returns '+extract_leading_character('เหง')
  assert extract_leading_character('โหนก') == 'น', 'โหนก returns '+extract_leading_character('โหนก')
  assert extract_leading_character('เหงา') == 'ง', 'เหงา returns '+extract_leading_character('เหงา')
  assert extract_leading_character('รอย') == 'ร', 'รอย returns '+extract_leading_character('รอย')
  assert extract_leading_character('อย่า') == 'ย', 'อย่า returns '+extract_leading_character('อย่า')
  assert extract_leading_character('หย่า') == 'ย', 'หย่า returns '+extract_leading_character('หย่า')
  assert extract_leading_character('หงอย') == 'ง', 'หงอย returns '+extract_leading_character('หงอย')
  # assert extract_leading_character('ห่วง') == 'ห', 'ห่วง returns '+extract_leading_character('ห่วง')
run_test_extract_leading_char()

In [0]:
subword_tokenize("ทรงพระเจริญ", engine="ssg")

['ทรง', 'พระ', 'เจริญ']

In [0]:
all_words = thai_stopwords().union(thai_words()).union(thai_negations()).union(countries()).union(provinces())#.union(thai_syllables())
all_words = set(all_words)
all_words_clean = set()
abbrv2word={}
for word in all_words:
  if '.' not in word:
    word= replace_um(word.strip())
    all_words_clean.add(word)
    abbrv = extract_abbreviation(word)
    if abbrv not in abbrv2word:
      abbrv2word[abbrv]=set()
    abbrv2word[abbrv].add(word)


In [0]:
get_leading_character(subword_tokenize('เลเซอร์', engine="ssg"))

'ลซ'

In [0]:
get_leading_character(subword_tokenize('พัฒนา', engine="ssg"))

'พน'

In [0]:
all_combination = []
def get_all_combination(abbrv,split_abbrv):
  for i in range(1,len(abbrv)+1):
    sub_abbrv= abbrv[:i]
    if sub_abbrv in abbrv2word:
      temp_abbrv = split_abbrv[:]
      temp_abbrv.append(sub_abbrv)
      if i < len(abbrv):
        get_full_words(abbrv[i:],temp_abbrv)
      else:
        all_combination.append(temp_abbrv)

def get_all_combination_recursive(abbrv_seq,remaining,index):
  if index== len(remaining)-1: 
    if remaining in abbrv2word:
      abbrv_seq.append(remaining)
      all_combination.append(abbrv_seq)
    return
  current_segment = remaining[:index+1]
  if current_segment in abbrv2word:
    temp_seq = abbrv_seq[:] + [current_segment]
    get_all_combination_recursive(temp_seq,remaining[index+1:],0)
  get_all_combination_recursive(abbrv_seq,remaining,index+1)




In [0]:
unigram_path = '/content/drive/My Drive/dataset/abbreviation_expansion/unigram.csv'
bigram_path = '/content/drive/My Drive/dataset/abbreviation_expansion/bigram.txt'
trigram_path = '/content/drive/My Drive/dataset/abbreviation_expansion/trigram.txt'
def read_file(file_path):
  with open(file_path, 'r',encoding='utf-8-sig') as f: #utf-8-sig is to omit byte order mark (BOM: \ufeff) at the first character
      return f.readlines()
def write_file(file_path,content_list):
  with open(file_path, 'w') as filehandle:
    for listitem in content_list:
        filehandle.write('%s\n' % str(listitem))
def get_unigram_frequency():
  unigram = {}
  for line in read_file(unigram_path):
    word,freq,per_mil = line.split(',')
    unigram[word] = int(freq)
  return unigram
def get_unigram_log_prob():
  unigram_log_prob={}
  for line in read_file(unigram_path):
    word,freq,per_mil = line.split(',')
    unigram_log_prob[word] = math.log(float(per_mil)/1000000)
  return unigram_log_prob
def calculate_oov_prob_from_unigram_with_min_freq(min_freq):
  sum_freq = 0
  sum_oov_freq = 0
  for line in read_file(unigram_path):
    word,freq,per_mil = line.split(',')
    freq = int(freq)
    sum_freq = sum_freq + freq
    if freq <= min_freq:
      sum_oov_freq = sum_oov_freq + freq
  return sum_oov_freq/sum_freq
def calculate_oov_prob_from_unigram():
  sum_freq = 0
  for line in read_file(unigram_path):
    word,freq,per_mil = line.split(',')
    freq = int(freq)
    sum_freq = sum_freq + freq
  return 1.0/sum_freq
def get_ngram_log_prob():
  unigram = {}
  bigram ={}
  count_beginning_tag = 0
  for line in read_file(bigram_path):
    first,second,freq = line.split('\t')
    if first not in bigram:
      bigram[first] = {}
      unigram[first] = 0
    if first == '<s/>':
      count_beginning_tag = count_beginning_tag +1
    bigram[first][second] = int(freq)
  unigram=get_unigram_frequency()
  unigram['<s/>'] = count_beginning_tag
  unigram_log_prob = get_unigram_log_prob()
  bigram_log_prob = calculate_bigram_log_prob(unigram,bigram)
  return unigram,unigram_log_prob,bigram,bigram_log_prob
def calculate_unigram_log_prob(unigram):
  unigram_log_prob={}
  sum_freq=0.0
  for word in unigram:
    sum_freq = sum_freq + unigram[word]
  for word in unigram:
    unigram_log_prob[word] = math.log(unigram[word]/sum_freq)
  return unigram_log_prob

def calculate_bigram_log_prob(unigram,bigram):
  bigram_log_prob={}
  for first_word in bigram:
    for second_word in bigram[first_word]:
      if first_word in unigram:
        freq_first_word = unigram[first_word]
        freq_first_and_second_word = bigram[first_word][second_word]
        p_second_given_first = freq_first_and_second_word/freq_first_word
        if first_word not in bigram_log_prob:
          bigram_log_prob[first_word] = {}
        bigram_log_prob[first_word][second_word] = math.log(p_second_given_first)
  return bigram_log_prob

def calculate_bigram_log_prob_with_add_1_smoothing(unigram,bigram):
  bigram_log_prob={}
  vocab_size = len(unigram.keys())
  for first_word in bigram:
    for second_word in bigram[first_word]:
      if first_word in unigram:
        freq_first_word = unigram[first_word]
        freq_first_and_second_word = bigram[first_word][second_word]
        p_second_given_first = (freq_first_and_second_word+1)/(freq_first_word+vocab_size)
        if first_word not in bigram_log_prob:
          bigram_log_prob[first_word] = {}
        bigram_log_prob[first_word][second_word] = math.log(p_second_given_first)
  return bigram_log_prob


In [0]:
unigram_frequency,unigram_log_prob,bigram,bigram_log_prob = get_ngram_log_prob()
VOCAB_SIZE = len(unigram_log_prob.keys())
OOV_LOG_PROB = math.log(calculate_oov_prob_from_unigram())

In [0]:
OOV_LOG_PROB

-17.323013268807358

In [0]:
def get_possible_words_from_abbrv(abbrv):
  if abbrv == '<s/>':
    return ['<s/>']
  elif abbrv in abbrv2word:
    return list(abbrv2word[abbrv])
  else:
    return []
def get_max_prob_word_sequence(states):
  current_word = '<s/>'
  prob = states[len(states.keys())-1][current_word][0]
  sequence = []
  for i in range(len(states.keys())-1,-1,-1):
    sequence = [current_word]+ sequence
    current_word  = states[i][current_word][1]
  return (prob,sequence)
def get_k_max_prob_word_sequence(states):
  k_max=[]
  top_k_list = states[len(states.keys())-1]['<s/>']
  for current_tuple in top_k_list:
    sequence = ['<s/>']
    current_prob = current_tuple[0]
    for i in range(len(states.keys())-2,-1,-1):
      current_word = current_tuple[1]
      current_index = current_tuple[2]
      current_tuple = states[i][current_word][current_index]
      sequence = [current_word]+ sequence
    k_max.append((current_prob,sequence))
  return k_max
def get_full_words_viterbi(combination,leading='<s/>'):

  states={0:
          {
            leading:(0,'')
           }
          }
  
  all_possible_first = [leading]
  combination = [leading]+combination
  for i in range(1,len(combination)):
    if i not in states:
      states[i]={}
    second_abbrv = combination[i]
    all_possible_second = get_possible_words_from_abbrv(second_abbrv)
    
    for first_word in all_possible_first:
      for second_word in all_possible_second:
        if second_word not in states[i]:
          states[i][second_word] = (-math.inf,'')
        if first_word in bigram_log_prob and second_word in bigram_log_prob[first_word]: 
          prob = bigram_log_prob[first_word][second_word]
        else:
          prob = OOV_LOG_PROB
          # prob = OOV_LOG_PROB
        # elif first_word in unigram_frequency:
        #   prob = math.log(1.0/(VOCAB_SIZE+ unigram_frequency[first_word])) #oov
        # else:
        #   prob = math.log(1.0/(VOCAB_SIZE+ 1)) #oov
        if (states[i-1][first_word][0] + prob) > states[i][second_word][0]:
          states[i][second_word] = (states[i-1][first_word][0] + prob,first_word)

    first_abbrv = second_abbrv
    all_possible_first = all_possible_second
  return get_max_prob_word_sequence(states)

def get_top_k_full_words_viterbi(combination,leading='<s/>',k=1):

  states={0:
          {
            leading:[(0,'',-1)] #(prob,previous_word,from_index)
           }
          }
  
  all_possible_first = [leading]
  combination = [leading]+combination
  for i in range(1,len(combination)):
    if i not in states:
      states[i]={}
    first_abbrv = combination[i-1]
    second_abbrv = combination[i]
    all_possible_first = list(states[i-1].keys())
    all_possible_second = get_possible_words_from_abbrv(second_abbrv)
    
    for first_word in all_possible_first:
      top_k_list = states[i-1][first_word]
      top_k_list.sort(key=lambda tup: tup[0],reverse=True)
      for j in range(len(top_k_list)):
        previous_state_prob = top_k_list[j][0]
        for second_word in all_possible_second:
          if second_word not in states[i]:
            states[i][second_word] = []
          if first_word in bigram_log_prob and second_word in bigram_log_prob[first_word]: 
            prob = bigram_log_prob[first_word][second_word]
          else:
            prob = OOV_LOG_PROB
            # prob = OOV_LOG_PROB
          # elif first_word in unigram_frequency:
          #   prob = math.log(1.0/(VOCAB_SIZE+ unigram_frequency[first_word])) #oov
          # else:
          #   prob = math.log(1.0/(VOCAB_SIZE+ 1)) #oov
          if len(states[i][second_word]) < k:
            heapq.heappush(states[i][second_word], (previous_state_prob+prob,first_word,j))
          else: 
            heapq.heappushpop(states[i][second_word], (previous_state_prob+prob,first_word,j))
    
  return get_k_max_prob_word_sequence(states)


In [0]:
get_top_k_full_words_viterbi(['ผ','น','ง','ร','จ','ต','ก','ม','<s/>'],leading='<s/>',k=3)

[(-31.952469467136428,
  ['<s/>', 'ผิด', 'ใน', 'แง่', 'ร้าย', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']),
 (-31.139427303950928,
  ['<s/>', 'ผล', 'ใน', 'แง่', 'ร้าย', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']),
 (-31.23851041136031,
  ['<s/>', 'ผม', 'ใน', 'แง่', 'ร้าย', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>'])]

In [0]:

def fill_abbreviation_viterbi(abbreviation,leading='<s/>',k=1):
  global all_combination
  all_combination = []
  all_full_words = []
  get_all_combination_recursive([],abbreviation,0)
  tokenized_leading = []
  #leading might contains several words, Tokenize it and send only the last word to the algorithm.
  if leading != '<s/>':
    tokenized_leading = word_tokenize(leading)
    leading = tokenized_leading.pop()
  for combination in all_combination:
    combination= combination + ['<s/>']
    full_words=[]
    top_k_full_word_list = get_top_k_full_words_viterbi(combination,leading,k)
    # insert tokenized_leading to the result
    for full_word in top_k_full_word_list:
      full_word = (full_word[0],tokenized_leading + full_word[1])
      full_words.append(full_word)

    all_full_words = all_full_words+ full_words
  all_full_words.sort(key=lambda tup: tup[0],reverse=True)
  return all_full_words

In [0]:
get_full_words_viterbi(['ผ','น','ง','ร','จ','ต','ก','ม','<s/>'],leading='<s/>')

(-31.139427303950928,
 ['<s/>', 'ผล', 'ใน', 'แง่', 'ร้าย', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>'])

In [0]:
get_top_k_full_words_viterbi(['ผ','น','ง','ร','จ','ต','ก','ม','<s/>'],leading='<s/>',k=3)

[(-31.952469467136428,
  ['<s/>', 'ผิด', 'ใน', 'แง่', 'ร้าย', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']),
 (-31.139427303950928,
  ['<s/>', 'ผล', 'ใน', 'แง่', 'ร้าย', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']),
 (-31.23851041136031,
  ['<s/>', 'ผม', 'ใน', 'แง่', 'ร้าย', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>'])]

In [0]:
if 'จินตนาการ' in abbrv2word['จตก']:
  abbrv2word['จตก'].remove('จินตนาการ') #จินตนาการ ตัดพยางค์ผิด จึงเอาออกจากพจนานุกรมคำย่อ
print(fill_abbreviation_viterbi('ผนงรจตกม',k=1)) 

[(-26.63360308256521, ['<s/>', 'ผล', 'ใน', 'งบ', 'รายจ่าย', 'ตาม', 'กฎหมาย', '<s/>']), (-28.93284268104506, ['<s/>', 'ผู้นำ', 'เงิน', 'รายจ่าย', 'ตาม', 'กฎหมาย', '<s/>']), (-29.750988330825002, ['<s/>', 'ผล', 'ใน', 'แง่', 'ร้าย', 'จริง', 'ตาม', 'กฎหมาย', '<s/>']), (-29.88885141495205, ['<s/>', 'ผ่าน', 'หน่วยงาน', 'รัฐ', 'จะ', 'ตาม', 'กฎหมาย', '<s/>']), (-30.46891683679882, ['<s/>', 'ผ่าน', 'หน่วยงาน', 'รัฐ', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']), (-31.139427303950928, ['<s/>', 'ผล', 'ใน', 'แง่', 'ร้าย', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']), (-31.203039870932365, ['<s/>', 'ผู้นำ', 'เงิน', 'เรา', 'จะ', 'ตาม', 'กฎหมาย', '<s/>']), (-31.302412934325705, ['<s/>', 'ผล', 'ใน', 'แง่', 'ร้าย', 'โจมตี', 'กัน', 'มาก', '<s/>']), (-31.72289155422237, ['<s/>', 'ผล', 'ใน', 'งบ', 'รายจ่าย', 'ต่าง', 'กัน', 'มาก', '<s/>']), (-31.78310529277913, ['<s/>', 'ผู้นำ', 'เงิน', 'เรา', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']), (-32.1635525202809, ['<s/>', 'ผล', 'ใน', 'แง่', 'ร้าย', 'จาก', 'โตเกียว', 'มิด', '<s/>']), (

In [0]:
print(fill_abbreviation_viterbi('ผนหรจตกม',k=1))

[(-27.87237660752656, ['<s/>', 'ผู้', 'นี้', 'ให้', 'รายจ่าย', 'ตาม', 'กฎหมาย', '<s/>']), (-27.91764999067034, ['<s/>', 'ผู้', 'นี้', 'ให้', 'เรา', 'จะ', 'ตาม', 'กฎหมาย', '<s/>']), (-28.345020271505092, ['<s/>', 'ผู้นำ', 'ให้', 'รายจ่าย', 'ตาม', 'กฎหมาย', '<s/>']), (-28.39029365464888, ['<s/>', 'ผู้นำ', 'ให้', 'เรา', 'จะ', 'ตาม', 'กฎหมาย', '<s/>']), (-28.497715412517106, ['<s/>', 'ผู้', 'นี้', 'ให้', 'เรา', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']), (-28.970359076495644, ['<s/>', 'ผู้นำ', 'ให้', 'เรา', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']), (-29.482888450224955, ['<s/>', 'ผล', 'ใน', 'ห้องเรียน', 'จะ', 'ตาม', 'กฎหมาย', '<s/>']), (-29.550523846056514, ['<s/>', 'ผู้', 'นี้', 'ให้', 'รู้จัก', 'แต่', 'กลับ', 'มา', '<s/>']), (-29.984984193116546, ['<s/>', 'ผู้', 'นี้', 'ให้', 'รู้จัก', 'ตรงกัน', 'หมด', '<s/>']), (-30.023167510035048, ['<s/>', 'ผู้นำ', 'ให้', 'รู้จัก', 'แต่', 'กลับ', 'มา', '<s/>']), (-30.06295387207172, ['<s/>', 'ผล', 'ใน', 'ห้องเรียน', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']), (-30.45

In [0]:
print(fill_abbreviation_viterbi('ฮรอ',k=1))

[(-7.452359399977921, ['<s/>', 'เฮโรอีน', '<s/>']), (-12.034798244485556, ['<s/>', 'เฮอ', 'ริ', 'โอ', '<s/>']), (-22.3883025253451, ['<s/>', 'เฮ้อ', 'ระอา', '<s/>']), (-23.988285971376417, ['<s/>', 'ฮีโร่', 'โอห์ม', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('งรจตกม', leading='ผู้นำ')) 

[(-25.96207871967089, ['ผู้นำ', 'เงิน', 'รายจ่าย', 'ตาม', 'กฎหมาย', '<s/>']), (-28.232275909558197, ['ผู้นำ', 'เงิน', 'เรา', 'จะ', 'ตาม', 'กฎหมาย', '<s/>']), (-28.812341331404962, ['ผู้นำ', 'เงิน', 'เรา', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']), (-31.051367191328055, ['ผู้นำ', 'เงิน', 'รายจ่าย', 'ต่าง', 'กัน', 'มาก', '<s/>']), (-31.171258822080446, ['ผู้นำ', 'เงิน', 'เรา', 'จะ', 'ต้องการ', 'มาก', '<s/>']), (-32.000548069828255, ['ผู้นำ', 'เงิน', 'หรือ', 'จุติ', 'กล่าว', 'มา', '<s/>']), (-38.695684759161345, ['ผู้นำ', 'เงิน', 'รายจ่าย', 'ตากลับ', 'มา', '<s/>']), (-39.52224078606867, ['ผู้นำ', 'เงิน', 'หรือ', 'จัดตั้ง', 'โกเมศ', '<s/>']), (-42.32539698566451, ['ผู้นำ', 'แง่ร้าย', 'จินต', 'กาล', 'มา', '<s/>']), (-43.073136201401915, ['ผู้นำ', 'แง่ร้าย', 'จ่าง', 'ตาม', 'กฎหมาย', '<s/>']), (-43.57578042546778, ['ผู้นำ', 'แง่ร้าย', 'จ่าง', 'ตระกูล', 'มอญ', '<s/>']), (-44.670257838971125, ['ผู้นำ', 'แง่ร้าย', 'จั้ง', 'แต่', 'กลับ', 'มา', '<s/>']), (-46.99937093809273, ['ผู้นำ', 'เงิน', 'หรือ', '

In [0]:
print(fill_abbreviation_viterbi('นงรจตกม', leading='ผู้')) 

[(-30.203956432610873, ['ผู้', 'นี้', 'งาน', 'รับจ้าง', 'ตาม', 'กฎหมาย', '<s/>']), (-31.789734365305883, ['ผู้', 'นี้', 'งั้น', 'เรา', 'จะ', 'ตาม', 'กฎหมาย', '<s/>']), (-32.369799787152644, ['ผู้', 'นี้', 'งั้น', 'เรา', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']), (-34.21161095213434, ['ผู้', 'นี้', 'งาน', 'รับจ้าง', 'ต่าง', 'กัน', 'มาก', '<s/>']), (-34.72871727782813, ['ผู้', 'นี้', 'งั้น', 'เรา', 'จะ', 'ต้องการ', 'มาก', '<s/>']), (-35.130928739940266, ['ผู้', 'ใน', 'แง่', 'ร้าย', 'โจมตี', 'กัน', 'มาก', '<s/>']), (-37.07567143246543, ['ผู้', 'หน่วยงาน', 'รัฐ', 'จะ', 'ตาม', 'กฎหมาย', '<s/>']), (-37.655736854312195, ['ผู้', 'หน่วยงาน', 'รัฐ', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']), (-39.43966747488903, ['ผู้', 'เนื้องอก', 'ร้าย', 'โจมตี', 'กัน', 'มาก', '<s/>']), (-39.5439588017369, ['ผู้', 'หน่วยงาน', 'รัฐ', 'จึง', 'ต้องการ', 'มาก', '<s/>']), (-39.63914899691677, ['ผู้', 'ในแง่', 'รองจ่าย', 'ตาม', 'กฎหมาย', '<s/>']), (-42.11333288288594, ['ผู้', 'นี้', 'งาน', 'รับจ้าง', 'ตากลับ', 'มา', '<s/>']),

In [0]:
print(fill_abbreviation_viterbi('นงรจตกม', leading='ผี',k=1)) 

[(-25.794786484724334, ['ผี', 'ใน', 'งบ', 'รายจ่าย', 'ตาม', 'กฎหมาย', '<s/>']), (-28.912171732984135, ['ผี', 'ใน', 'แง่', 'ร้าย', 'จริง', 'ตาม', 'กฎหมาย', '<s/>']), (-30.300610706110056, ['ผี', 'ใน', 'แง่', 'ร้าย', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']), (-30.46359633648483, ['ผี', 'ใน', 'แง่', 'ร้าย', 'โจมตี', 'กัน', 'มาก', '<s/>']), (-30.8840749563815, ['ผี', 'ใน', 'งบ', 'รายจ่าย', 'ต่าง', 'กัน', 'มาก', '<s/>']), (-31.32473592244003, ['ผี', 'ใน', 'แง่', 'ร้าย', 'จาก', 'โตเกียว', 'มิด', '<s/>']), (-37.07567143246543, ['ผี', 'หน่วยงาน', 'รัฐ', 'จะ', 'ตาม', 'กฎหมาย', '<s/>']), (-37.655736854312195, ['ผี', 'หน่วยงาน', 'รัฐ', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']), (-37.7093128231115, ['ผี', 'ใน', 'แง่', 'ร้าย', 'โจมตี', 'โกเมศ', '<s/>']), (-38.52839252421479, ['ผี', 'ใน', 'งบ', 'รายจ่าย', 'ตากลับ', 'มา', '<s/>']), (-39.43966747488903, ['ผี', 'เนื้องอก', 'ร้าย', 'โจมตี', 'กัน', 'มาก', '<s/>']), (-39.5439588017369, ['ผี', 'หน่วยงาน', 'รัฐ', 'จึง', 'ต้องการ', 'มาก', '<s/>']), (-39.6391489969167

In [0]:
print(fill_abbreviation_viterbi('งรจตกม', leading='ผัวหนู',k=1)) 

[(-27.995997510768213, ['ผัว', 'หนู', 'ง่วง', 'หรือ', 'จะ', 'ตาม', 'กฎหมาย', '<s/>']), (-28.576062932614978, ['ผัว', 'หนู', 'ง่วง', 'หรือ', 'จะ', 'ต้อง', 'กลับ', 'มา', '<s/>']), (-29.034571171298083, ['ผัว', 'หนู', 'งบ', 'รายจ่าย', 'ตาม', 'กฎหมาย', '<s/>']), (-29.653018150418834, ['ผัว', 'หนู', 'ง่วง', 'เริ่ม', 'จาก', 'โตเกียว', 'มิด', '<s/>']), (-30.579931305920553, ['ผัว', 'หนู', 'ง่วง', 'เริ่ม', 'จัดตั้ง', 'กรม', 'หมื่น', '<s/>']), (-34.123859642955246, ['ผัว', 'หนู', 'งบ', 'รายจ่าย', 'ต่าง', 'กัน', 'มาก', '<s/>']), (-37.05499173649806, ['ผัว', 'หนู', 'ง่วง', 'เริ่ม', 'จัดตั้ง', 'โกเมศ', '<s/>']), (-39.03880733897996, ['ผัว', 'หนู', 'เหงา', 'รู้จัก', 'ตรงกัน', 'หมด', '<s/>']), (-42.32539698566451, ['ผัว', 'หนู', 'แง่ร้าย', 'จินต', 'กาล', 'มา', '<s/>']), (-43.073136201401915, ['ผัว', 'หนู', 'แง่ร้าย', 'จ่าง', 'ตาม', 'กฎหมาย', '<s/>']), (-43.57578042546778, ['ผัว', 'หนู', 'แง่ร้าย', 'จ่าง', 'ตระกูล', 'มอญ', '<s/>']), (-44.670257838971125, ['ผัว', 'หนู', 'แง่ร้าย', 'จั้ง', 'แต่', 'กลับ

In [0]:
if 'ห้องสมุด' in abbrv2word['หส']:
  abbrv2word['หส'].remove('ห้องสมุด') #ห้องสมุด ตัดพยางค์ผิด จึงเอาออกจากพจนานุกรมคำย่อ
print(fill_abbreviation_viterbi('คหสต')) 

[(-11.925888654812185, ['<s/>', 'ความ', 'เห็น', 'ส่วนตัว', '<s/>']), (-14.167308846486453, ['<s/>', 'ความ', 'เห็น', 'สิ่ง', 'ต่าง', '<s/>']), (-14.32826486694159, ['<s/>', 'คิดเห็น', 'ส่วนตัว', '<s/>']), (-14.544481817361245, ['<s/>', 'ค้นหา', 'สิ่ง', 'ต่าง', '<s/>']), (-18.666880824339405, ['<s/>', 'คน', 'หัวสูง', 'ต้อง', '<s/>']), (-34.28613375489411, ['<s/>', 'คือ', 'เหตุส่วนตัว', '<s/>']), (-34.646026537614716, ['<s/>', 'ความเห็นส่วนตัว', '<s/>']), (-34.80017721744198, ['<s/>', 'คันหามเสือ', 'เตื้อ', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('หสต','ควาย')) 

[(-14.626237278642982, ['ควาย', 'แห่ง', 'สันติ', '<s/>']), (-16.957206258169023, ['ควาย', 'ให้', 'สิ่ง', 'ต่าง', '<s/>']), (-24.33790612431563, ['ควาย', 'หางเสียง', 'ตาม', '<s/>']), (-34.646026537614716, ['ควาย', 'เหตุส่วนตัว', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('จขกท')) 

[(-12.175096702223568, ['<s/>', 'เจ้าของ', 'กระท่อม', '<s/>']), (-12.414504879789842, ['<s/>', 'จะ', 'เข้า', 'กองทุน', '<s/>']), (-14.346058589001455, ['<s/>', 'จาก', 'ขอน', 'แก่น', 'แท้', '<s/>']), (-14.913202944075906, ['<s/>', 'เจ้าของ', 'กรรม', 'ที่', '<s/>']), (-15.186567703351155, ['<s/>', 'จาก', 'ขบวนการ', 'ที่', '<s/>'])]


In [0]:
if 'เสมอตีนนก' in abbrv2word['สตน']:
  abbrv2word['สตน'].remove('เสมอตีนนก') #ห้องสมุด ตัดพยางค์ผิด จึงเอาออกจากพจนานุกรมคำย่อ
print(fill_abbreviation_viterbi('ยสตน',k=1)) 

[(-11.880070425872885, ['<s/>', 'ยาเสพติด', 'นั้น', '<s/>']), (-14.76106465495753, ['<s/>', 'อย่าง', 'สูง', 'ต่อเนื่อง', '<s/>']), (-14.925413276030591, ['<s/>', 'อย่าง', 'สันติ', 'นั้น', '<s/>']), (-15.842462532663998, ['<s/>', 'ยี่สิบ', 'ตัว', 'นี้', '<s/>']), (-16.49399421384685, ['<s/>', 'อย่าง', 'สุด', 'ตัว', 'นี้', '<s/>']), (-22.807152659020304, ['<s/>', 'ยี่สิบ', 'ตีนนก', '<s/>']), (-35.3369152221936, ['<s/>', 'ยัง', 'สะเตงนอก', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('ยนน',k=1)) 

[(-6.3315199878797275, ['<s/>', 'อย่าง', 'แน่นอน', '<s/>']), (-7.815286251795134, ['<s/>', 'อย่างน้อย', 'หนึ่ง', '<s/>']), (-9.941345572850906, ['<s/>', 'ยัย', 'นี่', 'นา', '<s/>']), (-34.646026537614716, ['<s/>', 'หยามน้ำหน้า', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('อสกข',k=1)) 

[(-9.935918236558454, ['<s/>', 'อุตสาหกรรม', 'ขนาด', '<s/>']), (-11.727675753052543, ['<s/>', 'อุปสรรค', 'กีดขวาง', '<s/>']), (-13.6968878738074, ['<s/>', 'อุปสรรค', 'เกิด', 'ขึ้น', '<s/>']), (-14.996455977881178, ['<s/>', 'อีก', 'สิ่ง', 'กีดขวาง', '<s/>']), (-15.427393522625955, ['<s/>', 'อีก', 'สอง', 'เกิด', 'ขึ้น', '<s/>']), (-16.079109145759418, ['<s/>', 'อาจ', 'สะกิด', 'แขน', '<s/>']), (-35.58214250752371, ['<s/>', 'อาจ', 'สิ่งกีดขวาง', '<s/>'])]


In [0]:
if 'ไครสทเชิร์ช' in abbrv2word['คสช']:
  abbrv2word['คสช'].remove('ไครสทเชิร์ช') #ไครสทเชิร์ช ตัดพยางค์ผิด จึงเอาออกจากพจนานุกรมคำย่อ
print(fill_abbreviation_viterbi('คสช'))

[(-8.671246200057396, ['<s/>', 'ความ', 'สดชื่น', '<s/>']), (-9.999376861642002, ['<s/>', 'คุณ', 'สุ', 'ชาติ', '<s/>']), (-11.424788963546314, ['<s/>', 'โครงสร้าง', 'ชั้น', '<s/>']), (-34.646026537614716, ['<s/>', 'คำสาปแช่ง', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('อตอห')) 

[(-14.214635101507989, ['<s/>', 'อีก', 'โต๊ะ', 'อาหาร', '<s/>']), (-14.549198418411418, ['<s/>', 'เอา', 'ตัวเอง', 'ให้', '<s/>']), (-16.457461360532776, ['<s/>', 'อันตราย', 'อัน', 'ไหน', '<s/>']), (-17.328959908905908, ['<s/>', 'อาจ', 'ต้อง', 'เอา', 'ไหน', '<s/>']), (-22.028645629509203, ['<s/>', 'เอาแต่', 'อรหัง', '<s/>']), (-34.646026537614716, ['<s/>', 'ไอร์ตืองอ', 'แหง่', '<s/>']), (-35.58214250752371, ['<s/>', 'อาจ', 'โต๊ะอาหาร', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('สลทปปทดพน')) 

[(-27.403065613307565, ['<s/>', 'สัญลักษณ์', 'ที่', 'เป็น', 'ประเทศ', 'ด้อย', 'พัฒนา', '<s/>']), (-28.845248640199383, ['<s/>', 'สั่ง', 'ลงโทษ', 'เป็น', 'ประเทศ', 'ด้อย', 'พัฒนา', '<s/>']), (-29.465608971781943, ['<s/>', 'สี', 'เหลือง', 'ที่', 'เป็น', 'ประเทศ', 'ด้อย', 'พัฒนา', '<s/>']), (-29.617439193832645, ['<s/>', 'สัญลักษณ์', 'ที่', 'เปลี่ยนแปลง', 'ทาง', 'ด้าน', 'พัฒนา', '<s/>']), (-29.99508109084393, ['<s/>', 'ส่วน', 'หลัก', 'ทั่วไป', 'ประเทศ', 'ด้อย', 'พัฒนา', '<s/>']), (-30.206937867378155, ['<s/>', 'สั่ง', 'ลงโทษ', 'เปลี่ยนแปลง', 'ทาง', 'ด้าน', 'พัฒนา', '<s/>']), (-31.132307008219513, ['<s/>', 'สัญลักษณ์', 'ที่', 'เป็น', 'ไป', 'ที่', 'ด้อย', 'พัฒนา', '<s/>']), (-31.679982552307028, ['<s/>', 'สี', 'เหลือง', 'ที่', 'เปลี่ยนแปลง', 'ทาง', 'ด้าน', 'พัฒนา', '<s/>']), (-32.07662284285544, ['<s/>', 'สัญลักษณ์', 'ที่', 'เปลี่ยนแปลง', 'ทาง', 'ด้าน', 'พรรค์', 'นี้', '<s/>']), (-32.34160095597775, ['<s/>', 'สั่ง', 'ลงโทษ', 'ปรับ', 'เปลี่ยน', 'ที่', 'ด้อย', 'พัฒนา', '<s/>']), (-32.66612151

In [0]:
print(fill_abbreviation_viterbi('คหสต')) 

[(-11.925888654812185, ['<s/>', 'ความ', 'เห็น', 'ส่วนตัว', '<s/>']), (-14.167308846486453, ['<s/>', 'ความ', 'เห็น', 'สิ่ง', 'ต่าง', '<s/>']), (-14.32826486694159, ['<s/>', 'คิดเห็น', 'ส่วนตัว', '<s/>']), (-14.544481817361245, ['<s/>', 'ค้นหา', 'สิ่ง', 'ต่าง', '<s/>']), (-18.666880824339405, ['<s/>', 'คน', 'หัวสูง', 'ต้อง', '<s/>']), (-34.28613375489411, ['<s/>', 'คือ', 'เหตุส่วนตัว', '<s/>']), (-34.646026537614716, ['<s/>', 'ความเห็นส่วนตัว', '<s/>']), (-34.80017721744198, ['<s/>', 'คันหามเสือ', 'เตื้อ', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('จขกท',k=1)) 

[(-12.175096702223568, ['<s/>', 'เจ้าของ', 'กระท่อม', '<s/>']), (-12.414504879789842, ['<s/>', 'จะ', 'เข้า', 'กองทุน', '<s/>']), (-14.346058589001455, ['<s/>', 'จาก', 'ขอน', 'แก่น', 'แท้', '<s/>']), (-14.913202944075906, ['<s/>', 'เจ้าของ', 'กรรม', 'ที่', '<s/>']), (-15.186567703351155, ['<s/>', 'จาก', 'ขบวนการ', 'ที่', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('จขกท',k=3)) 

[(-12.175096702223568, ['<s/>', 'เจ้าของ', 'กระท่อม', '<s/>']), (-12.282985664234753, ['<s/>', 'เจ้าของ', 'กระทู้', '<s/>']), (-12.342593726749072, ['<s/>', 'เจ้าของ', 'กองทุน', '<s/>']), (-12.414504879789842, ['<s/>', 'จะ', 'เข้า', 'กองทุน', '<s/>']), (-13.10990601538669, ['<s/>', 'จะ', 'เข้า', 'กรุงเทพ', '<s/>']), (-13.462473724896316, ['<s/>', 'จึง', 'เข้า', 'กองทุน', '<s/>']), (-14.346058589001455, ['<s/>', 'จาก', 'ขอน', 'แก่น', 'แท้', '<s/>']), (-14.836507663980255, ['<s/>', 'จะ', 'เข้า', 'กับ', 'ที่', '<s/>']), (-14.913202944075906, ['<s/>', 'เจ้าของ', 'กรรม', 'ที่', '<s/>']), (-15.162202695348105, ['<s/>', 'เจ้าของ', 'เก่า', 'ที่', '<s/>']), (-15.186567703351155, ['<s/>', 'จาก', 'ขบวนการ', 'ที่', '<s/>']), (-15.229268495071967, ['<s/>', 'จึง', 'ขอ', 'กราบ', 'เท้า', '<s/>']), (-15.43953073621404, ['<s/>', 'เจ้าของ', 'ก่อน', 'ที่', '<s/>']), (-15.615956707466731, ['<s/>', 'จะ', 'เข้าเกณฑ์', 'ทหาร', '<s/>']), (-15.64960120791928, ['<s/>', 'จาก', 'ของเก่า', 'ที่', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('ผนปทหต',k=1)) 

[(-18.941789043439105, ['<s/>', 'ผู้นำ', 'ประเทศ', 'ให้', 'ตาย', '<s/>']), (-21.990988084066093, ['<s/>', 'ผล', 'ใน', 'ประเทศ', 'ให้', 'ตาย', '<s/>']), (-22.649531479159055, ['<s/>', 'ผู้', 'นี้', 'เป็น', 'ที่', 'ใหญ่โต', '<s/>']), (-22.859801755391725, ['<s/>', 'ผู้นำ', 'ไป', 'ที่', 'ใหญ่โต', '<s/>']), (-23.453648379111833, ['<s/>', 'ผู้นำ', 'ไป', 'ทำให้', 'ตน', '<s/>']), (-23.750529231621286, ['<s/>', 'ผม', 'นำ', 'ไป', 'ทำให้', 'ตน', '<s/>']), (-24.282891680345127, ['<s/>', 'ผู้นำ', 'ประเทศ', 'หันตรา', '<s/>']), (-24.564346071469174, ['<s/>', 'แผง', 'หน้าปัด', 'ที่', 'ใหญ่โต', '<s/>']), (-25.101854526304155, ['<s/>', 'ผู้', 'นี้', 'เป็น', 'ที่', 'ให้', 'ตาย', '<s/>']), (-25.126508290826013, ['<s/>', 'ผู้นำ', 'ไป', 'เที่ยว', 'ไหน', 'ต่อ', '<s/>']), (-26.854313222661826, ['<s/>', 'ผิว', 'หน้าเป็น', 'ที', 'เห', 'ตู', '<s/>']), (-27.332090720972115, ['<s/>', 'ผล', 'ใน', 'ประเทศ', 'หันตรา', '<s/>']), (-34.49262443661234, ['<s/>', 'ผิว', 'หน้าเป็น', 'ทุกแห่ง', 'ต่าง', '<s/>']), (-34.942024

In [0]:
print(fill_abbreviation_viterbi('ควย',k=1)) 

[(-7.8389303322163535, ['<s/>', 'ความ', 'ห่วงใย', '<s/>']), (-8.478125954856914, ['<s/>', 'คลื่นวิทยุ', '<s/>']), (-11.433066284212709, ['<s/>', 'คิด', 'ว่า', 'อยู่', '<s/>']), (-11.480870691696337, ['<s/>', 'ความหวัง', 'อยู่', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('กกต',k=1)) 

[(-7.753792641541903, ['<s/>', 'กิจกรรม', 'ต่าง', '<s/>']), (-8.720233240938702, ['<s/>', 'การ', 'ก่อตั้ง', '<s/>']), (-9.263011882853586, ['<s/>', 'กัลกัตตา', '<s/>']), (-9.74667155960639, ['<s/>', 'การ', 'ก่อ', 'ตัว', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('อห',leading='<s/>',k=2)) 

[(-4.388297302589001, ['<s/>', 'อาหาร', '<s/>']), (-6.038803427960931, ['<s/>', 'โอ้', 'โห', '<s/>']), (-7.269582217830298, ['<s/>', 'อรหันต์', '<s/>']), (-7.518698746981279, ['<s/>', 'อัน', 'ไหน', '<s/>'])]


In [0]:
print(fill_abbreviation_viterbi('สวดค'))

[(-14.213907446335316, ['<s/>', 'แสดง', 'ว่า', 'ด้วย', 'ค่ะ', '<s/>']), (-16.030545538378796, ['<s/>', 'สภาวะ', 'เดิม', 'คือ', '<s/>']), (-16.747729480021235, ['<s/>', 'แสดง', 'ว่า', 'ดาวเคราะห์', '<s/>']), (-18.23254698044048, ['<s/>', 'ส่วน', 'วัสดุ', 'คือ', '<s/>']), (-21.596793766227844, ['<s/>', 'สุวรรณ', 'ได้ความ', '<s/>'])]


In [0]:
subword_tokenize("พยายาม", engine="ssg")

['พยา', 'ยาม']

In [0]:
subword_tokenize("จินตนาการ", engine="ssg")

['จิน', 'ตนา', 'การ']

In [0]:
subword_tokenize("สวัสดี", engine="ssg")

['สวัส', 'ดี']

In [0]:
print(fill_abbreviation_viterbi('ต',leading ='เดือด',k=30)) 

[(-17.477163948634615, ['เดือด', 'เตื้อ', '<s/>']), (-17.610695341259138, ['เดือด', 'ตรอน', '<s/>']), (-17.62722464321035, ['เดือด', 'ตราด', '<s/>']), (-17.62918015593324, ['เดือด', 'ตรัง', '<s/>']), (-17.728478376915522, ['เดือด', 'โตนด', '<s/>']), (-17.728478376915522, ['เดือด', 'ตวัก', '<s/>']), (-17.859317977874333, ['เดือด', 'ตรึม', '<s/>']), (-17.862009769540045, ['เดือด', 'เตย', '<s/>']), (-17.929149072377673, ['เดือด', 'เติร์ก', '<s/>']), (-17.986307486217623, ['เดือด', 'ตฤณ', '<s/>']), (-18.007649759699394, ['เดือด', 'ตุ่น', '<s/>']), (-18.009380762381923, ['เดือด', 'เตียน', '<s/>']), (-18.011374479051362, ['เดือด', 'ตัน', '<s/>']), (-18.0161604493673, ['เดือด', 'ตรำ', '<s/>']), (-18.031198326731843, ['เดือด', 'ต๋อม', '<s/>']), (-18.078680806348636, ['เดือด', 'ตุ้บ', '<s/>']), (-18.083299752204933, ['เดือด', 'ต๊อก', '<s/>']), (-18.111470629171627, ['เดือด', 'ตรอม', '<s/>']), (-18.12650850653617, ['เดือด', 'ตุง', '<s/>']), (-18.149691841991825, ['เดือด', 'ตุบ', '<s/>']), (-18.1